In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

import re
import unicodedata
import html
from nltk.tokenize import casual_tokenize
# from nltk.corpus import stopwords

In [ ]:
# Definindo funções auxiliares

# *************************************************************
# Limpeza de texto
# *************************************************************

MD_LINK_RE   = re.compile(r'\[([^\]]+)\]\((?:https?://|www\.)[^\s)]+\)', re.I)
SCHEME_URL_RE= re.compile(r'(?i)\b(?:https?://|http://|ftp://|www\.)[^\s<>()]+\b')
BARE_DOMAIN_RE = re.compile(
    r'(?i)\b(?!\S+@)(?:[a-z0-9-]+\.)+(?:[a-z]{2,})(?:/[^\s<>()]*)?\b'
)

def clean_text(text: str) -> str:
    """Clean HTML, emojis/odd symbols, control chars; tokenize with NLTK casual and join."""
    if text is None or (isinstance(text, float) and pd.isna(text)):
        return ""
    if not isinstance(text, str):
        text = str(text)
        
    text = unicodedata.normalize("NFKC", text)
    text = html.unescape(text)
    text = re.sub(r"<[^>]+>", " ", text)
    text = re.sub(r"[\r\n\t\f\v]+", " ", text)
    
    # removendo alguns links
    text = MD_LINK_RE.sub(r"\1", text)
    text = SCHEME_URL_RE.sub(" ", text)
    text = BARE_DOMAIN_RE.sub(" ", text)
    
    # removendo não alfanuméricos, exceto pontuação básica
    text = re.sub(r"[^0-9A-Za-zÀ-ÖØ-öø-ÿçÇ\s\.\,\!\?\:\;\'\"\(\)\-\_\/]", " ", text)
    
    # tokenização com casual tokenizer
    tokens = casual_tokenize(text, reduce_len=True, strip_handles=True, preserve_case=False)
    text = " ".join(tokens)
    
    # removendo espaços extras
    text = re.sub(r"\s{2,}", " ", text).strip()
    return text

# *************************************************************
# Criando arquivo de treinamento para spaCy
# *************************************************************
def find_tokens(input_filename, output_filename, num_rows_to_process):
    df = pd.read_csv(input_filename)

    # Lista de operadoras para procurar
    operators = ["vivo", "oi", "tim", "claro"]
    train_data = []

    # Garante que não vamos processar mais linhas do que as que existem no arquivo.
    actual_rows = min(num_rows_to_process, len(df))

    for i in range(0, actual_rows):
        
        text = df.loc[i, "text"]
        
        # Check se o texto é uma string válida
        if not isinstance(text, str):
            continue

        entities = []

        for op in operators:
            # Usa limites de palavra para encontrar apenas palavras inteiras, ignorando o case
            for match in re.finditer(r'\b' + re.escape(op) + r'\b', text, re.IGNORECASE):
                start, end = match.span()
                entities.append((start, end, "OPERADORA"))
                
        entities.sort(key=lambda x: x[0])
        
        # Adiciona o texto e suas entidades à lista de dados de treinamento
        train_data.append((text, entities))

    # Escrevendo em arquivo python
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write("# Arquivo de treinamento para o spaCy\n")
        f.write(f"# Contém as primeiras {len(train_data)} linhas do arquivo de entrada.\n\n")
        f.write("train_data = [\n")
        for item in train_data:
            # Formata cada item como uma tupla
            f.write(f"    {repr(item)},\n")
        f.write("]\n")

    print(f"Processo concluído.")
    print(f"O arquivo de treinamento '{output_filename}' foi gerado com {len(train_data)} linhas.")


# *************************************************************
# Downsampling com proveniência
# *************************************************************
def downsample_all_with_provenance(groups: dict[str, dict[str, pd.DataFrame]],
                                   random_state: int = 42,
                                   join: str = "inner"):

    # descobrir o menor tamanho entre todos os dataframes
    sizes = { (g,k): len(df) for g, d in groups.items() for k, df in d.items() }
    if not sizes:
        return {}, 0, pd.DataFrame()
    min_n = min(sizes.values())

    # para cada dataframe, fazer downsample para min_n (sem reposição)
    balanced_groups: dict[str, dict[str, pd.DataFrame]] = {g: {} for g in groups}
    sampled_frames = []
    for g, mapping in groups.items():
        for k, df in mapping.items():
            
            df_with_src = df.copy()
            df_with_src["grupo"] = g
            df_with_src["operadora"] = k
            
            if len(df_with_src) > min_n:
                df_with_src = df_with_src.sample(n=min_n, replace=False, random_state=random_state)
            balanced_groups[g][k] = df_with_src
            sampled_frames.append(df_with_src)

    # concatenar todos os dataframes amostrados
    all_concat = pd.concat(sampled_frames, ignore_index=True, join=join)
    return balanced_groups, min_n, all_concat


# *************************************************************
# Amostragem balanceada
# *************************************************************
def stratified_fix_split_with_test(df: pd.DataFrame,
                                   key_cols=('grupo', 'operadora'),
                                   per_subset_per_group=50,
                                   random_state=42):
    """
    Returns train_df, dev_df, test_df.
    - train/dev: exactly `per_subset_per_group` rows per subgroup each (no replacement)
    - test: everything else left over from `df`
    """
    rng = random_state
    groups = df.groupby(list(key_cols), sort=False)
    need = 2 * per_subset_per_group

    train_parts, dev_parts = [], []
    used_idx = []

    for grp_key, gdf in groups:
        n = len(gdf)
        if n < need:
            raise ValueError(
                f"Not enough rows in subgroup {grp_key}: need {need}, have {n}."
            )

        gdf = gdf.sample(frac=1, random_state=rng)

        train_g = gdf.iloc[:per_subset_per_group].copy()
        dev_g   = gdf.iloc[per_subset_per_group:need].copy()

        train_parts.append(train_g)
        dev_parts.append(dev_g)

        used_idx.extend(train_g.index.tolist())
        used_idx.extend(dev_g.index.tolist())

    train_df = pd.concat(train_parts, ignore_index=False)
    dev_df   = pd.concat(dev_parts,   ignore_index=False)

    # test = df original exceto os índices usados em train/dev
    test_df = df.drop(index=list(set(used_idx)))

    # shuffle adicional
    train_df = train_df.sample(frac=1, random_state=rng+1).reset_index(drop=True)
    dev_df   = dev_df.sample(frac=1,   random_state=rng+2).reset_index(drop=True)
    test_df  = test_df.sample(frac=1,  random_state=rng+3).reset_index(drop=True)

    return train_df, dev_df, test_df

In [ ]:
# carregando os dados de cada operadora
operadoras = ['oi', 'tim', 'vivo', 'claro']

df_operadora = {op: pd.read_csv(f"data/operadora_{op}_ptbr.csv").drop_duplicates("text") for op in operadoras}
df_geral     = {op: pd.read_csv(f"data/{op}_geral.csv").drop_duplicates("text")           for op in operadoras}

# realizando downsampling, balancearmento e amostragem
groups = {"operadora": df_operadora, "geral": df_geral}
balanced_groups, min_n, all_8_balanced = downsample_all_with_provenance(groups, random_state=42, join="inner")
print(f"Global downsample target (rows per DF): {min_n}")

train_df, dev_df, test_df = stratified_fix_split_with_test(
    all_8_balanced, key_cols=('grupo','operadora'), per_subset_per_group=50, random_state=42
)
print(train_df.shape, dev_df.shape, test_df.shape)

# sanity checks
assert len(train_df) == 400 and len(dev_df) == 400
print(train_df.groupby(['grupo','operadora']).size())
print(dev_df.groupby(['grupo','operadora']).size())
print(test_df.groupby(['grupo','operadora']).size())

# limpando os textos
train_df["text"] = train_df["text"].apply(clean_text)
dev_df["text"] = dev_df["text"].apply(clean_text)
test_df["text"] = test_df["text"].apply(clean_text)


/tmp/ipykernel_4814/3373588540.py:4: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_operadora = {op: pd.read_csv(f"data/operadora_{op}_ptbr.csv").drop_duplicates("text") for op in operadoras}
/tmp/ipykernel_4814/3373588540.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geral     = {op: pd.read_csv(f"data/{op}_geral.csv").drop_duplicates("text")           for op in operadoras}
/tmp/ipykernel_4814/3373588540.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geral     = {op: pd.read_csv(f"data/{op}_geral.csv").drop_duplicates("text")           for op in operadoras}


Global downsample target (rows per DF): 9271
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74168 entries, 0 to 74167
Data columns (total 38 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   activities                            18 non-null     object 
 1   content_type                          73741 non-null  object 
 2   creation_time                         74168 non-null  object 
 3   id                                    74167 non-null  float64
 4   is_branded_content                    74168 non-null  bool   
 5   lang                                  74168 non-null  object 
 6   link_attachment.caption               9076 non-null   object 
 7   link_attachment.description           8217 non-null   object 
 8   link_attachment.link                  9258 non-null   object 
 9   link_attachment.name                  9336 non-null   object 
 10  match_type                           

In [ ]:
# salvando como csv
train_df.to_csv('dataset/train.csv', index=False)
dev_df.to_csv('dataset/dev.csv', index=False)
test_df.to_csv('dataset/test.csv', index=False)

In [ ]:
# criando arquivos de treinamento para spaCy, apenas primeiras 400 linhas cada
find_tokens('dataset/train.csv', 'train_v2.py', 400)
find_tokens('dataset/dev.csv', 'dev_v2.py', 400)
find_tokens('dataset/test.csv', 'test_v2.py', 400)

Processo concluído.
O arquivo de treinamento 'train_v2.py' foi gerado com 400 linhas.
Processo concluído.
O arquivo de treinamento 'dev_v2.py' foi gerado com 400 linhas.
Processo concluído.
O arquivo de treinamento 'test_v2.py' foi gerado com 400 linhas.


## Treinamento do modelo

In [3]:
from spacy.tokens import DocBin

from dataset.train import train_data
from dataset.dev import dev_data
from dataset.test import test_data

In [ ]:
nlp = spacy.blank("pt")  # Modelo vazio em português

doc_bin_train = DocBin()

for text, entities in train_data:
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in entities:
        span = doc.char_span(start, end, label=label)
        if span:
            ents.append(span)
    doc.ents = ents
    doc_bin_train.add(doc)

doc_bin_train.to_disk("train.spacy")

doc_bin_dev = DocBin()

for text, entities in dev_data:
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in entities:
        span = doc.char_span(start, end, label=label)
        if span:
            ents.append(span)
    doc.ents = ents
    doc_bin_dev.add(doc)

doc_bin_dev.to_disk("dev.spacy")

#  Script para treinar o modelo spaCy usando os arquivos gerados:
#  python3 -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
model = "last"

# 1. Carregar o modelo treinado que foi salvo na pasta 'output/model-best'
nlp_ner = spacy.load("./output/model-" + model)

print("--- Avaliação do Modelo no Dataset de Desenvolvimento ---\n")

# Lista para armazenar cada linha de resultado para o Excel
results_for_excel = []

# 2. Iterar sobre os dados de desenvolvimento para comparar o previsto com o correto
for example in test_data:
    text = example[0]
    true_annotations = example[1]
    
    true_entities_list = sorted([text[start:end] for start, end, label in true_annotations])
    doc = nlp_ner(text)
    predicted_entities_list = sorted([ent.text for ent in doc.ents])
    is_equal = (true_entities_list == predicted_entities_list)
    
    # Adiciona os resultados formatados à lista
    results_for_excel.append({
        "text": text,
        "actual_annotations": true_annotations,
        "predicted_annotations": [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents],
        "actual_entities": ", ".join(true_entities_list),
        "predicted_entities": ", ".join(predicted_entities_list),
        "is_equal": is_equal,
        "model": model
    })

# 3. Gerar o Arquivo Excel
if results_for_excel:
    output_filename = "model_evaluation_output_"+ model +"_v2.xlsx"
    
    print("\nGerando o arquivo Excel com os resultados...")

    results_df = pd.DataFrame(results_for_excel)
    results_df.to_excel(output_filename, index=False, engine='xlsxwriter')
    
    print(f"✅ Processo concluído! Os resultados da avaliação foram salvos em '{output_filename}'")
else:
    print("\nNenhum dado foi processado. O arquivo Excel não foi gerado.")


--- Avaliação do Modelo no Dataset de Desenvolvimento ---


Gerando o arquivo Excel com os resultados...
✅ Processo concluído! Os resultados da avaliação foram salvos em 'model_evaluation_output_last_v2.xlsx'
